# Library Imports

In [ ]:
%%capture

import bokeh
import pandas
import qgrid
import numpy

from bokeh.io import output_file
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.layouts import layout
from bokeh.models import ColumnDataSource
from bokeh.models import CustomJS
from bokeh.models import EqHistColorMapper
from bokeh.models import HoverTool
from bokeh.models import Legend
from bokeh.models import LinearColorMapper
from bokeh.models import RadioButtonGroup
from bokeh.models import Slider
from bokeh.plotting import figure

from bokeh.palettes import Viridis8 as palette
from bokeh.palettes import Category20 as Cat20
palette = tuple(reversed(palette))
cat_palette = Cat20[20]

bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as counties

# Load Datasets
Look at data/README.md for more information on the datasets

In [ ]:
# Load autism prevalence data
autism = pandas.read_csv("data/ADV_AllData.csv")
# State codes are lower case in the other tables
state_list = [s.lower() for s in set(autism['Location'])]
# Remove not needed states
state_list.remove('ak')
state_list.remove('hi')

# Load census data
census = pandas.read_csv(
    "data/census.txt", dtype={'year': int, 'fips': str, 'age': int, 'pop': int})
# group by year and county
tot_pop = census.groupby(['year', 'fips'])

# Load vaccine data
name = ['polio', 'dtap', 'MMR', 'Hib', 'varicelle',
        'hepatB', 'hepatA','PCV', 'rotavirus']
coverage = []
for nn in name:
    coverage += [pandas.read_csv("data/"+ nn +"_row.txt")]

# Adjust the counties dictionary    
counties = {
    code: county for code, county in counties.items() if county["state"] in state_list
}
statefull_list = list(set([counties[county_id]['detailed name'].split(', ')[1]
                            for county_id in counties]))

# Inspect Tables

## Autism Prevalence

## Vaccination Coverage

In [ ]:
# Print prevalence data
table_viz = qgrid.show_grid(autism, show_toolbar=True)
table_viz

In [ ]:
# Print vaccine data
table_viz = qgrid.show_grid(pandas.concat(coverage), show_toolbar=True)
table_viz

# Interactive Maps

## Plotting Tools

In [ ]:
county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]
county_names = [county['name'] for county in counties.values()]

button_code = """
console.log('radio_button_group: active=' + this.active, this.toString())
var data = source.data;
var values = data['values']
var newvalues = data['values_' + this.labels[this.active]]
for (var i = 0; i < values.length; i++) {
    values[i] = newvalues[i]
}
source.change.emit();
"""


def InteractiveMapPlot(year, values, maxim, minim,
                       name, mapper=EqHistColorMapper):
    output_notebook()
    color_mapper = mapper(
        palette=palette, high=maxim, low=minim, low_color="#FFFFFF")

    data = dict(x=county_xs,
                y=county_ys,
                name=county_names)
    for y in year:
        data.update({"values_%d" % (y): values[y]})
    data.update({"values": values[year[0]]})
    source = ColumnDataSource(data)

    TOOLS = "pan,wheel_zoom,reset,hover,save"
    p = figure(plot_width=1000, plot_height=800,
               title="U.S. %s"%name, tools=TOOLS,
               x_axis_location=None, y_axis_location=None,
               tooltips=[("Name", "@name"),
                         (name, "@values"),
                         ("(Long, Lat)", "($x, $y)")
                        ])
    p.grid.grid_line_color = None
    p.hover.point_policy = "follow_mouse"

    patch_renderer = p.patches(
        'x', 'y', source=source, fill_alpha=0.7,
        line_color="black", line_width=0.5,
        fill_color={'field': 'values', 'transform': color_mapper})

    callback = CustomJS(args=dict(source=source), code=button_code)
    labels = [str(y) for y in year]
    radio_button_group = RadioButtonGroup(labels=labels, active=0)
    radio_button_group.js_on_click(callback)

    show(layout([[p], [radio_button_group]]))
    output_file("US_%s.html"%name)


In [ ]:
year = sorted(list(set(census['year'])))
county_pop = {}
for y in year:
    county_pop[y] = []
    for county_id in counties:
        curr_fips = "%02d" % (county_id[0]) + "%03d" % (county_id[1])
        if (y, curr_fips) in tot_pop.groups:
            county_pop[y] += [
                float(tot_pop.get_group((y, curr_fips))['pop'].sum())]
        else:
            county_pop[y] += [0.]

# retieve the max and min values
maxim = census.groupby(['year', 'fips'])['pop'].sum().max()
minim = census.groupby(['year', 'fips'])['pop'].sum().min()

InteractiveMapPlot(year, county_pop, maxim, minim,
                   'Population', mapper=EqHistColorMapper)

## Autism Prevalence

In [ ]:
# Could be one of ['medi', 'addm', 'sped', 'nsch']
prevalence = autism[autism['Source'] == 'medi']
year = sorted(list(set(prevalence['Year'])))

county_autism = {}
for y in year:
    county_autism[y] = []
    for county_id in counties:
        curr_state = (counties[county_id]['state']).upper()
        prev = prevalence[
            (prevalence['Year'] == y) &
            (prevalence['Location'] == curr_state)]['Prevalence']
        if len(prev) and not(numpy.isnan(prev.values[0])):
            assert len(prev) == 1
            county_autism[y] += [prev.values[0]]
        else:
            county_autism[y] += [0.]

# retieve the max and min values
maxim = prevalence['Prevalence'].max()
minim = prevalence['Prevalence'].min()

InteractiveMapPlot(year, county_autism, maxim, minim,
                   'Autism Prevalence', mapper=LinearColorMapper)

## Vaccine Coverage

In [ ]:
coverage_concat = pandas.concat(coverage)
dtap = coverage_concat[(coverage_concat['dose'] == '4dosesDTaPVaccinationAge') & 
                       (coverage_concat['category'] == '19-35 Months')]
year = sorted(list(set(dtap['year'])))

county_coverage = {}
for y in year:
    county_coverage[y] = []
    for county_id in counties:
        curr_state = (counties[county_id]['detailed name']).split(', ')[1]
        cov = dtap[
            (dtap['year'] == y) &
            (dtap['location'] == curr_state)]['coverage']
        if len(cov) and not(numpy.isnan(cov.values[0])):
            assert len(cov) == 1
            county_coverage[y] += [cov.values[0]]
        else:
            county_coverage[y] += [0.]

# retieve the max and min values
maxim = dtap['coverage'].max()
minim = dtap['coverage'].min()

InteractiveMapPlot(year, county_coverage, maxim, minim,
                   'Vaccination Coverage', mapper=LinearColorMapper)

# Longitudinal Analysis

## Plotting Tools

In [ ]:
dash  = ['solid', 'dashed', 'dotted']
seed = 20


def InteractiveLinePlot(year, states, low,
                        mean, high, name, visible_every):
    output_notebook()
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    p = figure(plot_width=1000, plot_height=1300, tools=TOOLS)
    numpy.random.seed(seed)
    order = numpy.random.permutation(len(states))
    
    legend_it = []
    for idx, s in enumerate(states):
        data = dict(year = year[s] + year[s][::-1] + year[s] + year[s][::-1],
                    values = low[s] + mean[s][::-1] + high[s] + mean[s][::-1])
        source = ColumnDataSource(data)
        c = p.patch(x='year', y='values', source=source, level='underlay',
                    fill_alpha=.3, fill_color=cat_palette[order[idx]%20],
                    line_width=1.2, line_alpha=1, line_color=cat_palette[order[idx]%20],
                    line_dash=dash[order[idx]//20]) 
        if idx%visible_every !=0:
            c.visible = False
        legend_it.append((s, [c]))
    p.xgrid[0].grid_line_color=None
    p.ygrid[0].grid_line_alpha=0.5
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = name
    legend = Legend(items=legend_it)
    legend.click_policy="hide"
    p.add_layout(legend, 'right')
    show(p)

## Autism Prevalence

In [ ]:
# Could be one of ['medi', 'addm', 'sped', 'nsch']
prevalence = autism[autism['Source'] == 'sped']
year = sorted(list(set(prevalence['Year'])))
states = sorted(list(set(prevalence['Location'])))

state_autism = {}
state_low = {}
state_high = {}
state_year = {}
for s in states:
    state_autism[s] = []
    state_low[s] = []
    state_high[s] = []
    state_year[s] = []
    for y in year:
        current = prevalence[
            (prevalence['Year'] == y) &
            (prevalence['Location'] == s)]
        prev = current['Prevalence']
        prev_min = current['Lower CI']
        prev_max = current['Upper CI']
        if len(prev) and not(numpy.isnan(prev.values[0])):
            assert len(prev) == 1
            state_autism[s] += [prev.values[0]]
            state_low[s] += [prev_min.values[0]]
            state_high[s] += [prev_max.values[0]]
            state_year[s] += [y]

InteractiveLinePlot(state_year, states, state_low, state_autism, state_high, 'Autism Prevalence', 2)

## Vaccination Coverage

In [ ]:
coverage_concat = pandas.concat(coverage)
dtap = coverage_concat[(coverage_concat['dose'] == '4dosesDTaPVaccinationAge') & 
                       (coverage_concat['category'] == '19-35 Months')]
year = sorted(list(set(dtap['year'])))
states = sorted(statefull_list)

state_coverage = {}
state_low = {}
state_high = {}
state_year = {}
for s in states:
    state_coverage[s] = []
    state_low[s] = []
    state_high[s] = []
    state_year[s] = []
    for y in year:
        current = dtap[
            (dtap['year'] == y) &
            (dtap['location'] == s)]
        prev = current['coverage']
        prev_min = current['low']
        prev_max = current['high']
        if len(prev) and not(numpy.isnan(prev.values[0])):
            assert len(prev) == 1
            state_coverage[s] += [prev.values[0]]
            state_low[s] += [prev_min.values[0]]
            state_high[s] += [prev_max.values[0]]
            state_year[s] += [y]

InteractiveLinePlot(state_year, states, state_low, state_coverage, state_high, 'Vaccine Coverage', 10)


# Additional Data

## U.S. Census

In [ ]:
# Print census data
table_viz = qgrid.show_grid(census, show_toolbar=True)
table_viz

In [ ]:
year = sorted(list(set(census['year'])))
county_pop = {}
for y in year:
    county_pop[y] = []
    for county_id in counties:
        curr_fips = "%02d" % (county_id[0]) + "%03d" % (county_id[1])
        if (y, curr_fips) in tot_pop.groups:
            county_pop[y] += [
                float(tot_pop.get_group((y, curr_fips))['pop'].sum())]
        else:
            county_pop[y] += [0.]

# retieve the max and min values
maxim = census.groupby(['year', 'fips'])['pop'].sum().max()
minim = census.groupby(['year', 'fips'])['pop'].sum().min()

InteractiveMapPlot(year, county_pop, maxim, minim,
                   'Population', mapper=EqHistColorMapper)